In [1]:
import json, openai
import os, sys, re
from openai import OpenAI
from tqdm.notebook import tqdm
import time
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
from config.keys import OPENAI_API_KEY

In [25]:
class Generator:
    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.json_formatter = "Return the response in RFC8259 compliant JSON according to the ResponseFormat schema with no other text."
        self.message = [{
            "role": "system",
            "content": "You are a cyber-security programmer that can detect line numbers from the contract based on the instruction."
        }]
        self.output_formatter = """
This should instruct the model to output exactly the vulnerability lines, ensuring it doesn't output extra lines or large ranges that contain unrelated code.

Response Schema:
 [
    {
      "start_line": <exact_start_line_number>,
      "end_line": <exact_end_line_number>,
      "code": [
        "vulnerable line 1",
        "vulnerable line 2",
        "... (and so on)"
      ]
    }
 ]
 
** Do not use ```json or any other extra texts in the output. Include only the list of detected lines as the schema.
"""
        self.user_prefix = """You are given a record from a dataset containing smart contract vulnerability analyses. Each record contains an "id", "prompt", and "completion". The "prompt" includes both the instructions and the smart contract code snippet, while the "completion" provides the vulnerability analysis.

Your task is to extract the exact lines of code that are vulnerable from the smart contract code. If the contract is vulnerable, output the minimal range of lines (with exact start and end line numbers) and the code lines themselves in the JSON Response Schema format provided below. If the contract is safe, do not output any JSON (i.e. return an empty result).

Instructions:
- Extract the smart contract code from the prompt.
- If the completion analysis indicates that the contract is vulnerable (i.e. it does not state that the contract is safe), locate the vulnerable lines of code precisely.
- Do not include any extra commentary or unrelated code; only include the vulnerable lines.
"""
    def get_user_message(self, dataset_output, contract):
        self.user_content = f"""
{self.user_prefix}

This is the helping document to find the lines of vulnerable codes.
Guideline:
{dataset_output}

Smart contract code:
{contract}

Additional Note:
- Only output the minimal range of code lines that are directly vulnerable.
- Do not include any commentary or unrelated code.
- Follow the JSON Response Schema exactly:
---
{self.output_formatter}

###
"""
        self.user_message = {"role": "user", "content": self.user_content}
    def create_prompt(self, dataset_output, contract):
        self.get_user_message(dataset_output, contract)
        self.message.append(self.user_message)
        
    def generate(self):
        completion = self.client.chat.completions.create(
          model="gpt-4o-mini",
          messages = self.message,
          temperature=0.1,
          max_tokens=3200,
          top_p=1.,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        answer = completion.choices[0].message.content
        return answer, completion


In [26]:
dataset_name = "train_TrustLLM"
output_dir = os.path.join("..", "..", "data", "processed_data", dataset_name)
os.makedirs(output_dir, exist_ok=True)

contracts_dir = os.path.join(output_dir, "contracts")
os.makedirs(contracts_dir, exist_ok=True)

locs_dir = os.path.join(output_dir, "LOCs")
os.makedirs(locs_dir, exist_ok=True)

raw_dir = os.path.join("..", "..", "data", "dataset", "raw")
dataset_path = os.path.join(raw_dir, f"{dataset_name}.json")

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset file not found at: {dataset_path}")

In [27]:
with open(dataset_path, "r", encoding="utf-8") as file:
    data = json.load(file)
print("Total records:", len(data))
#print(data[1]["completion"])

Total records: 20885


In [28]:
ids = [record.get("id") for record in data]
if len(ids) == len(set(ids)):
    print("All record ids are unique.")
else:
    print("There are duplicate record ids.")

There are duplicate record ids.


In [29]:
end_contract = 6
vul_idx = 0  # Counter for vulnerable smart contracts

for idx, record in enumerate(tqdm(data[:end_contract], desc="Processing Records")):
    prompt_text = record.get("prompt", "")
    completion_text = record.get("completion", "")
    
    # Extract smart contract code from the prompt.
    # This regex now matches any code block enclosed in triple backticks regardless of the language tag.
    code_match = re.search(r"```[^\n]*\n(.*?)\n```", prompt_text, re.DOTALL)
    if code_match:
        contract_code = code_match.group(1).strip()
    else:
        print(f"Record {idx}: No smart contract code found in prompt.")
        continue
    
    # Skip safe contracts
    if "appears to be safe" in completion_text.lower():
        print(f"Record {idx} is marked as safe. Skipping vulnerability extraction.")
        continue
    # Otherwise, if it does not mention "the issue", assume there is no clear vulnerability description.
    if "the issue" not in completion_text.lower():
        print(f"Record {idx} does not clearly indicate a vulnerability. Skipping vulnerability extraction.")
        continue

    # Save the vulnerable smart contract code to a .sol file using vul_idx for numbering
    sol_filepath = os.path.join(contracts_dir, f"{vul_idx}.sol")
    with open(sol_filepath, 'w', encoding='utf-8') as f:
        f.write(contract_code)
    
    # Add line numbers to the contract code for vulnerability mapping
    lines = contract_code.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    numbered_contract = "\n".join(numbered_lines)
    
    print(f"Extracting vulnerability for Record {idx} (vulnerable contract index: {vul_idx})...")
    generator = Generator()
    # Use the 'completion' as the guideline for vulnerability extraction
    generator.create_prompt(completion_text, numbered_contract)
    start_time = time.time()
    answer, _ = generator.generate()
    print(f"Vulnerability extraction for Record {idx} completed in {round(time.time() - start_time, 2)} seconds")
    
    # Parse the answer as JSON and save the vulnerability details if extraction was successful.
    try:
        vulnerability_data = json.loads(answer)
    except Exception as e:
        print(f"Error parsing vulnerability JSON for Record {idx}: {e}")
        continue
    
    json_filepath = os.path.join(locs_dir, f"{vul_idx}.json")
    with open(json_filepath, "w", encoding="utf-8") as json_file:
        json.dump(vulnerability_data, json_file, indent=4)
    print(f"Saved vulnerability data for Record {idx} to {json_filepath}")
    
    vul_idx += 1

print("Process completed!")

Processing Records:   0%|          | 0/6 [00:00<?, ?it/s]

Record 0 is marked as safe. Skipping vulnerability extraction.
Record 1 is marked as safe. Skipping vulnerability extraction.
Record 2 is marked as safe. Skipping vulnerability extraction.
Record 3 is marked as safe. Skipping vulnerability extraction.
Record 4 is marked as safe. Skipping vulnerability extraction.
Extracting vulnerability for Record 5 (vulnerable contract index: 0)...
Vulnerability extraction for Record 5 completed in 1.6 seconds
Saved vulnerability data for Record 5 to ../../data/processed_data/train_TrustLLM/LOCs/0.json
Process completed!


In [36]:
print(generator.message[1]['content'])


You are given a record from a dataset containing smart contract vulnerability analyses. Each record contains an "id", "prompt", and "completion". The "prompt" includes both the instructions and the smart contract code snippet, while the "completion" provides the vulnerability analysis.

Your task is to extract the exact lines of code that are vulnerable from the smart contract code. If the contract is vulnerable, output the minimal range of lines (with exact start and end line numbers) and the code lines themselves in the JSON Response Schema format provided below. If the contract is safe, do not output any JSON (i.e. return an empty result).

Instructions:
- Extract the smart contract code from the prompt.
- If the completion analysis indicates that the contract is vulnerable (i.e. it does not state that the contract is safe), locate the vulnerable lines of code precisely.
- Do not include any extra commentary or unrelated code; only include the vulnerable lines.


This is the helpin

In [31]:
print(answer)

[
    {
        "start_line": 5,
        "end_line": 5,
        "code": [
            "if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); }"
        ]
    }
]


In [52]:
text = "Below is an instruction that describes a reasoning task.\nCarefully assess the contributing factors and their interplay. Utilize the label name to form a coherent reasoning.\n### Instruction:\nPlease analyze the code function and its dependencies, including both incoming and outgoing calls. Considering the label name, identify the underlying cause.\n### Input:\nThe method FuseTokenAdapterV1 in the smart contract wrap\n```Solidiy\nfunction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }\n```\n## Dependency Overview:\n### As a Caller:\nFunctions called by FuseTokenAdapterV1:\n```\nSafeERC20.safeTransferFrom\nFuseTokenAdapterV1.address\nSafeERC20.safeApprove\nFuseTokenAdapterV1.IERC20\nFuseTokenAdapterV1.ICERC20\nFuseTokenAdapterV1.FuseError\nSafeERC20.safeTransfer\nmodifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized(\"Not alchemist\"); } _; }\n```\n### Response:\nGiven that the code is labeled vulnerable, let's determine the reason by breaking down the process."
# solidity_code = re.search(r'Solidiy(.*?)', text, re.DOTALL)

import re

# text = """
# ```solidity

# some texts

# ```solidity

# The rest
# """

# Regex pattern to match Solidity code blocks
pattern = r"```Solidiy\n(.*?)\n```"

matches = re.findall(pattern, text, re.DOTALL)

# Print extracted Solidity code blocks
for i, match in enumerate(matches, 1):
    print(match)


function wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }


In [54]:
print('''unction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }\n```\n## Dependency Overview:\n### As a Caller:\nFunctions called by FuseTokenAdapterV1:\n```\nSafeERC20.safeTransferFrom\nFuseTokenAdapterV1.address\nSafeERC20.safeApprove\nFuseTokenAdapterV1.IERC20\nFuseTokenAdapterV1.ICERC20\nFuseTokenAdapterV1.FuseError\nSafeERC20.safeTransfer\nmodifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized(\"Not alchemist\"); } _; }\n''')

unction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }
```
## Dependency Overview:
### As a Caller:
Functions called by FuseTokenAdapterV1:
```
SafeERC20.safeTransferFrom
FuseTokenAdapterV1.address
SafeERC20.safeApprove
FuseTokenAdapterV1.IERC20
FuseTokenAdapterV1.ICERC20
FuseTokenAdapterV1.FuseError
SafeERC20.safeTransfer
modifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized("Not alchemist"); } _; }

